# Peer-graded Assignment: Build a Regression Model in Keras

### A. Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the 
train_test_split
helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

### Import Dataset

In [5]:
import pandas as pd
import numpy as np

#import dataset
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Split data into predictors and target


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [7]:
n_cols = predictors.shape[1] # number of predictors

## Build a Neural Network


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense


# Step 1: Randomly split the data into training and test sets
errors = []

for _ in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)
    # Convert Pandas DataFrames to NumPy arrays
    predictors_train = predictors_train.values
    predictors_test = predictors_test.values
    target_train = target_train.values
    target_test = target_test.values

    # Step 2: Build the neural network
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors_train.shape[1],)))
    model.add(Dense(1))  # Output layer with 1 node for regression
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 3: Train the model
    model.fit(predictors_train, target_train, epochs=50, verbose=0)

    # Step 4: Evaluate the model on the test data
    predictions = model.predict(predictors_test)
    mse = mean_squared_error(target_test, predictions)
    errors.append(mse)

# Step 5: Report the mean and standard deviation of the mean squared errors
mean_mse = np.mean(errors)
std_mse = np.std(errors)

print(f"Mean Squared Error: {mean_mse}")
print(f"Standard Deviation of MSE: {std_mse}")


10/10 [==============================] - 0s 304us/step
Mean Squared Error: 294.6865900859241
Standard Deviation of MSE: 285.7902308414322


## B. Normalize the data (5 marks) 

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Assuming 'concrete_data' is your DataFrame containing the data

# Step 1: Randomly split the data into training and test sets
errors_normalized = []

for _ in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)

    # Normalize the predictors using StandardScaler
    scaler = StandardScaler()
    predictors_train_scaled = scaler.fit_transform(predictors_train)
    predictors_test_scaled = scaler.transform(predictors_test)

    # Convert to NumPy arrays
    target_train_np = target_train.values
    target_test_np = target_test.values

    # Step 2: Build the neural network
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors_train_scaled.shape[1],)))
    model.add(Dense(1))  # Output layer with 1 node for regression
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 3: Train the model
    model.fit(predictors_train_scaled, target_train_np, epochs=50, verbose=0)

    # Step 4: Evaluate the model on the test data
    predictions = model.predict(predictors_test_scaled)
    mse = mean_squared_error(target_test_np, predictions)
    errors_normalized.append(mse)

# Step 5: Report the mean and standard deviation of the mean squared errors for normalized data
mean_mse_normalized = np.mean(errors_normalized)
std_mse_normalized = np.std(errors_normalized)

print(f"Mean Squared Error (Normalized): {mean_mse_normalized}")
print(f"Standard Deviation of MSE (Normalized): {std_mse_normalized}")


10/10 [==============================] - 0s 348us/step
Mean Squared Error (Normalized): 347.5443665902096
Standard Deviation of MSE (Normalized): 81.59316526961535


It looks like normalizing the data had an impact on the mean squared error. The mean squared error increased from 294.69 (in the original, non-normalized case) to 347.54 in the normalized case. The standard deviation of the mean squared error also decreased from 285.79 to 81.59.

In general, normalizing data is a common practice in machine learning to ensure that features are on a similar scale, which can improve the training stability and convergence of the neural network. However, the effect on the performance (in terms of mean squared error) can vary depending on the specific characteristics of the data and the model.

## C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Assuming 'concrete_data' is your DataFrame containing the data

# Step 1: Randomly split the data into training and test sets
errors_normalized = []

for _ in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)

    # Normalize the predictors using StandardScaler
    scaler = StandardScaler()
    predictors_train_scaled = scaler.fit_transform(predictors_train)
    predictors_test_scaled = scaler.transform(predictors_test)

    # Convert to NumPy arrays
    target_train_np = target_train.values
    target_test_np = target_test.values

    # Step 2: Build the neural network
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors_train_scaled.shape[1],)))
    model.add(Dense(1))  # Output layer with 1 node for regression
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 3: Train the model
    model.fit(predictors_train_scaled, target_train_np, epochs=100, verbose=0)

    # Step 4: Evaluate the model on the test data
    predictions = model.predict(predictors_test_scaled)
    mse = mean_squared_error(target_test_np, predictions)
    errors_normalized.append(mse)

# Step 5: Report the mean and standard deviation of the mean squared errors for normalized data
mean_mse_normalized = np.mean(errors_normalized)
std_mse_normalized = np.std(errors_normalized)

print(f"Mean Squared Error (Normalized): {mean_mse_normalized}")
print(f"Standard Deviation of MSE (Normalized): {std_mse_normalized}")


10/10 [==============================] - 0s 355us/step
Mean Squared Error (Normalized): 166.83116811074854
Standard Deviation of MSE (Normalized): 16.580480432717486


It appears that increasing the number of epochs from 50 to 100 had a noticeable effect on the mean squared error. The mean squared error decreased substantially from 347.54 (with 50 epochs) to 166.83 (with 100 epochs), suggesting that the model may benefit from more training epochs.

This behavior is not uncommon. Increasing the number of epochs allows the model more opportunities to adjust its weights and learn from the data. However, it's essential to monitor for signs of overfitting, where the model may start fitting the training data too closely and not generalize well to new, unseen data.

## D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Assuming 'concrete_data' is your DataFrame containing the data

# Step 1: Randomly split the data into training and test sets
errors_three_layers = []

for _ in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)

    # Normalize the predictors using StandardScaler
    scaler = StandardScaler()
    predictors_train_scaled = scaler.fit_transform(predictors_train)
    predictors_test_scaled = scaler.transform(predictors_test)

    # Convert to NumPy arrays
    target_train_np = target_train.values
    target_test_np = target_test.values

    # Step 2: Build the neural network with three hidden layers
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors_train_scaled.shape[1],)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))  # Output layer with 1 node for regression
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 3: Train the model
    model.fit(predictors_train_scaled, target_train_np, epochs=50, verbose=0)

    # Step 4: Evaluate the model on the test data
    predictions = model.predict(predictors_test_scaled)
    mse = mean_squared_error(target_test_np, predictions)
    errors_three_layers.append(mse)

# Step 5: Report the mean and standard deviation of the mean squared errors for three hidden layers
mean_mse_three_layers = np.mean(errors_three_layers)
std_mse_three_layers = np.std(errors_three_layers)

print(f"Mean Squared Error (Three Hidden Layers): {mean_mse_three_layers}")
print(f"Standard Deviation of MSE (Three Hidden Layers): {std_mse_three_layers}")


10/10 [==============================] - 0s 332us/step
Mean Squared Error (Three Hidden Layers): 126.68835618480627
Standard Deviation of MSE (Three Hidden Layers): 18.220522734685826


The mean squared error with three hidden layers (126.69) is significantly lower than the mean squared error from Step B with normalization (347.54). This suggests that the model with three hidden layers performed better in terms of prediction accuracy compared to the model with a single hidden layer.

In comparison with Step C, where the mean squared error was 166.83 (with 100 epochs and normalization), the mean squared error further decreased to 126.69 when three hidden layers with 10 nodes each and ReLU activation function were introduced.

It indicates that the increased model complexity introduced by adding more hidden layers and nodes allowed the neural network to capture more intricate patterns in the data, leading to improved performance.